# Merge and Groupby
Duncan Callaway

This notebook gives an introduction to using Pandas' `merge` and `groupby` methods.  

In [1]:
import pandas as pd
import numpy as np

## Row and column labels
The columns are identified with a list of values.  Let's look at the fruit data set again:

In [2]:
fruit_info_df = pd.read_csv('fruit_info.csv', index_col= False)
fruit_info_df

,fruit,color,weight
0,apple,red,120
1,banana,yellow,150
2,orange,orange,250
3,raspberry,pink,15


In [3]:
fruit_info_df.columns

Index(['fruit', 'color', 'weight'], dtype='object')

The rows are similarly labeled:

In [4]:
fruit_info_df.index

RangeIndex(start=0, stop=4, step=1)

## Merging
Lets make another data frame and tack it on to the first

In [5]:
price_df = pd.DataFrame({'price':[0.5, 0.65, 1, 0.15],
                        'frut':['apple', 'banana', 'orange', 'rasberry']})
price_df

,price,frut
0,0.50,apple
1,0.65,banana
2,1.00,orange
3,0.15,rasberry


In [6]:
fruit_info_df

,fruit,color,weight
0,apple,red,120
1,banana,yellow,150
2,orange,orange,250
3,raspberry,pink,15


Now let's blindly merge:

In [7]:
pd.merge(price_df,fruit_info_df)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

What went wrong?

First, we didn't spell fruit correctly.  Two ways to fix.  First, specify the columns directly:

In [8]:
pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit')

,price,frut,fruit,color,weight
0,0.50,apple,apple,red,120
1,0.65,banana,banana,yellow,150
2,1.00,orange,orange,orange,250


Second, fix the spelling and *don't* tell pandas.  In this case pandas works to figure out what's in common.

In [9]:
price_df.columns[0]='fruit'

TypeError: Index does not support mutable operations

Bummer!  Can't mutate index values.  What to do?

In [10]:
col_list = list(price_df.columns)
col_list

['price', 'frut']

In [11]:
col_list[1] = 'fruit'

In [12]:
price_df.columns = col_list
price_df

,price,fruit
0,0.50,apple
1,0.65,banana
2,1.00,orange
3,0.15,rasberry


In [13]:
pd.merge(fruit_info_df,price_df)

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00


Note we can use different syntax:

In [14]:
fruit_info_df.merge(price_df)

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00


Now we're still missing raspberries -- why?

Again, spelling error in the new frame.  Let's fix:

In [15]:
price_df.loc[3,'fruit'] = 'raspberry'

Note we could change individual entries in the data frame itself.  They are mutable.

In [16]:
fruit_info_df.merge(price_df)

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00
3,raspberry,pink,15,0.15


Another few things to takeaway from this
1. Merge can be brutal.  That is, it'll drop data without telling you.  BUT that's if we use the default 'inner' merge.  In a few lecture we'll talk about alternative ways to merge that are a little less draconian.
2. It's important to review your results.  How many rows do you expect?  How many do you actually get?  Did something important get chucked out?  The ensuing solutions are the non-glamorous tasks of data cleaning.

Note, there are other commands -- `join`, `concat`, and these do similar things to `merge`.

I've found merge seems to work well for most purposes.

FWIW, `pd.concat` seems to be a little more brute force -- requires more careful syntax, but likely does unexpected things less often once you understand the syntax.

In [17]:
merged_df = fruit_info_df.merge(price_df)
merged_df

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00
3,raspberry,pink,15,0.15


We can streamline by replacing the index number with the fruit column.  

What's the `inplace` command for?  It means the re-defined dataframe is assigned to the original name.  This is advantageous in memory constrained situations.  

In [18]:
merged_df.set_index('fruit', inplace = True)
merged_df

,color,weight,price
fruit,,,
apple,red,120,0.50
banana,yellow,150,0.65
orange,orange,250,1.00
raspberry,pink,15,0.15


## Multilevel indexing
We can also assign "multilevel" column or row names, like so:

In [19]:
levels = [('categorical', 'color'),('quantitative', 'weight'),('quantitative','price')]
levels

[('categorical', 'color'),
 ('quantitative', 'weight'),
 ('quantitative', 'price')]

Note the  use of tuples (sets of values in parentheses) in setting up multiindex.  This will come again later.  

In [20]:
merged_df.columns = pd.MultiIndex.from_tuples(levels)
merged_df

categorical quantitative      
                color       weight price
fruit                                   
apple             red          120  0.50
banana         yellow          150  0.65
orange         orange          250  1.00
raspberry        pink           15  0.15

Now we have categories and subcategories of columns:

In [21]:
merged_df['quantitative']

,weight,price
fruit,,
apple,120,0.50
banana,150,0.65
orange,250,1.00
raspberry,15,0.15


Note, we can also drop and add things.  With multilevel indexing things get a little tricky.  

First, we can drop everything from the top level:

In [22]:
merged_test_df = merged_df.drop(columns=[('quantitative',)], axis = 1)
merged_test_df

,categorical
,color
fruit,
apple,red
banana,yellow
orange,orange
raspberry,pink


Note that I put the column identifier inside the parens, like a tuple, but it's not essential there.

However if we want to drop only a column from the second level, we get an error without the tuple syntax:

In [23]:
merged_test_df = merged_df.drop(columns=[('quantitative','price')], axis = 1)
merged_test_df

,categorical,quantitative
,color,weight
fruit,,
apple,red,120
banana,yellow,150
orange,orange,250
raspberry,pink,15


We can also drop rows: 

In [24]:
merged_df.drop(index=[('apple')], axis = 0, inplace = True)
merged_df

categorical quantitative      
                color       weight price
fruit                                   
banana         yellow          150  0.65
orange         orange          250  1.00
raspberry        pink           15  0.15

Note indexing multilevels with `.loc` gets a little tricky.  The thing to keep in mind is that you're working with tuples in each index location:

In [25]:
merged_df.loc['banana', ('quantitative', 'price')]

0.65

If you leave an entry of the tuple empty you get all values.  

In [26]:
merged_df.loc['banana', ('quantitative', )]

weight     150
price     0.65
Name: banana, dtype: object

You can also loop through the columns of the multilevel data frame like this: 

In [27]:
for i, j in merged_df:
    print(merged_df.loc['banana', (i, j)])

yellow
150
0.65


Some added thoughts:
1. Multilevel indexing works for columns and index
2. It can be a powerful way to summarize your data and quickly reference subsets of it.
4. However it can also be a colossal pain in the rear -- indexing with multilevel is often very hard to parse and debug.

## Groupby
(these notes adapted from Spring 2018 DS100 notebook)

First, let's have another look at today's power point file.  Now we'll learn about how groupby works.

Back to the notebook, let's make a toy DF (example taken from Wes McKinney's [Python for Data Analysis](http://proquest.safaribooksonline.com.libproxy.berkeley.edu/book/programming/python/9781491957653):

In [28]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.240236,0.303708
1,a,two,0.053173,-0.341313
2,b,one,-0.150036,-1.689679
3,b,two,1.590772,1.207901
4,a,one,-2.295662,0.214001


Let's group just the `data1` column by the `key1` column. A call to [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) does that.  

Note, the syntax is to begin by invoking the portion of the dataframe we want to group (here, `df['data1']`), then we apply the groupby method with the portion of hte dataframe we want to group on (here `df['key1']`)

What is the object that results?

In [29]:
grouped = df['data1'].groupby(df['key1'])
grouped

As we see, it's not simply a new DataFrame. Instead, it's an object, in this case `SeriesGroupBy`.  We'll see in a moment that if we group many columns of data we get a `DataFrameGroupBy` object.

To look inside we need to use different syntax.  The specific thing we're looking for are the groups of the object...but let's tab in to the grouped object to see what's there.

In [30]:
grouped.groups

{'a': Int64Index([0, 1, 4], dtype='int64'),
 'b': Int64Index([2, 3], dtype='int64')}

That gave us the groups (a and b) and the indices of elements in the groups, but nothing else. 

You can see this structure looks like a dict.  a and b are the keys, and the data are lists associated with each key -- the values.  

But the `grouped` object is capable of making computations across all groups -- this is where it gets powerful.   

We can try things like `.count()`, `.min()` and `.mean()`.  

Notice if you don't put the parens after the method, pandas returns information about what the method does, but not it's actual output.  

In [31]:
grouped.aggregate(min)

key1
a   -2.295662
b   -0.150036
Name: data1, dtype: float64

There are a number of functions you can pass in to ``aggregate``, like `sum` and `max`.

But it can be informative to look at what's inside. We can iterate over a `groupby` object, as we iterate we get pairs of `(name, group)`, where the `group` is either a `Series` or a `DataFrame`, depending on whether the `groupby` object is a `SeriesGroupBy` (as above) or a `DataFrameGroupBy` (see below).

Something quirky to note about the interaction between the grouped object and the for loop structure below: we're going to define variables `name` and `group` as being things in `grouped`.  But there are no `name` or `group` attributes associated with the `grouped` object.  These 

In [32]:
from IPython.display import display  # like print, but for complex objects

for name, group in grouped:
    print('Name:', name)
    display(group)

Name: a


0    0.240236
1    0.053173
4   -2.295662
Name: data1, dtype: float64

Name: b


2   -0.150036
3    1.590772
Name: data1, dtype: float64

We can group on multiple keys, and the result is grouping by tuples:

In [33]:
g2 = df['data1'].groupby([df['key1'], df['key2']])
g2

In [34]:
g2.groups

{('a', 'one'): Int64Index([0, 4], dtype='int64'),
 ('a', 'two'): Int64Index([1], dtype='int64'),
 ('b', 'one'): Int64Index([2], dtype='int64'),
 ('b', 'two'): Int64Index([3], dtype='int64')}

Now we have a groupby object that has tuples as the keys.  

Let's look at the dataframe again, for a reminder:

In [35]:
df

,key1,key2,data1,data2
0,a,one,0.240236,0.303708
1,a,two,0.053173,-0.341313
2,b,one,-0.150036,-1.689679
3,b,two,1.590772,1.207901
4,a,one,-2.295662,0.214001


In [36]:
g2.mean()

key1  key2
a     one    -1.027713
      two     0.053173
b     one    -0.150036
      two     1.590772
Name: data1, dtype: float64

We can also group the entire dataframe -- not just one column of it -- on a single key. This results in a `DataFrameGroupBy` object as the result:

In [37]:
k1g = df.groupby('key1')
k1g

In [38]:
k1g.groups

{'a': Int64Index([0, 1, 4], dtype='int64'),
 'b': Int64Index([2, 3], dtype='int64')}

That output actually looks a lot like the output when we were only grouping one of the columns of the dataframe.  But there is actually more information in the group itself.  

In [39]:
k1g.mean()

,data1,data2
key1,,
a,-0.667418,0.058799
b,0.720368,-0.240889


Let's look at what's inside of k1g:

In [40]:
for n, g in k1g:
    print('name:', n)
    display(g)

name: a


,key1,key2,data1,data2
0,a,one,0.240236,0.303708
1,a,two,0.053173,-0.341313
4,a,one,-2.295662,0.214001


name: b


,key1,key2,data1,data2
2,b,one,-0.150036,-1.689679
3,b,two,1.590772,1.207901


Where did column `key2` go in the mean above? It's a *nuisance column*, which gets automatically eliminated from an operation where it doesn't make sense (such as a numerical mean).

### Grouping over a different dimension

Above, we've been grouping data along the rows, using column keys as our selectors.  

But we can also group along the *columns*, 

What's even more cool?  We can group by *data type*.

Here we'll group along columns, by data type:

In [41]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [42]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    display(group)

float64


,data1,data2
0,0.240236,0.303708
1,0.053173,-0.341313
2,-0.150036,-1.689679
3,1.590772,1.207901
4,-2.295662,0.214001


object


,key1,key2
0,a,one
1,a,two
2,b,one
3,b,two
4,a,one


## Using groupby to re-ask our question
Which hour had the lowest average wind production?

In [43]:
cds = pd.read_csv('CAISO_2017to2018_stack.csv', index_col= 0)

In [44]:
cds.head()

,Source,MWh
2017-08-29 00:00:00,GEOTHERMAL,1181
2017-08-29 00:00:00,BIOMASS,340
2017-08-29 00:00:00,BIOGAS,156
2017-08-29 00:00:00,SMALL HYDRO,324
2017-08-29 00:00:00,WIND TOTAL,1551


It will help to have a column of hour of day values:

In [45]:
cds_time = pd.to_datetime(cds.index)
type(cds_time)

pandas.core.indexes.datetimes.DatetimeIndex

Let's add that list of values into the data frame.

In [46]:
cds['hour'] = cds_time.hour

In [47]:
cds.head(10)

,Source,MWh,hour
2017-08-29 00:00:00,GEOTHERMAL,1181,0
2017-08-29 00:00:00,BIOMASS,340,0
2017-08-29 00:00:00,BIOGAS,156,0
2017-08-29 00:00:00,SMALL HYDRO,324,0
2017-08-29 00:00:00,WIND TOTAL,1551,0
2017-08-29 00:00:00,SOLAR PV,0,0
2017-08-29 00:00:00,SOLAR THERMAL,0,0
2017-08-29 01:00:00,GEOTHERMAL,1182,1
2017-08-29 01:00:00,BIOMASS,338,1
2017-08-29 01:00:00,BIOGAS,156,1


Now do the grouping.

See if you can do it yourself: we want to group MWh values by source AND hour.

In [48]:
cds_grouped = cds['MWh'].groupby([cds['Source'],cds['hour']])

Now we can see *all* the means for all sources and hours.

Didn't need to do any fancy logical indexing or looping!

In [49]:
cds_grouped.mean()

Source      hour
BIOGAS      0        225.591781
            1        225.964384
            2        225.953425
            3        225.887671
            4        225.753425
                       ...     
WIND TOTAL  19      2181.361644
            20      2229.408219
            21      2231.687671
            22      2220.109589
            23      2216.526027
Name: MWh, Length: 168, dtype: float64

Now it would be nice to see that information in a dataframe, wouldn't it?

In [50]:
averages = pd.DataFrame(cds_grouped.mean())

In [51]:
averages

MWh
Source     hour             
BIOGAS     0      225.591781
           1      225.964384
           2      225.953425
           3      225.887671
           4      225.753425
...                      ...
WIND TOTAL 19    2181.361644
           20    2229.408219
           21    2231.687671
           22    2220.109589
           23    2216.526027

[168 rows x 1 columns]

And lo and behold, we have a multilevel index for the rows!

In [52]:
averages.loc[('WIND TOTAL',),:]

,MWh
hour,
0,2173.268493
1,2120.778082
2,2051.832877
3,1973.969863
4,1881.463014
5,1772.484932
6,1646.630137
7,1490.194521
8,1363.402740


But now we can look at other sources

In [53]:
averages.loc[('SMALL HYDRO',),:]

,MWh
hour,
0,330.824658
1,322.421918
2,318.249315
3,316.909589
4,322.254795
5,375.180822
6,426.931507
7,422.564384
8,376.813699


In [54]:
plt.plot(averages.loc[('SMALL HYDRO',),:])

NameError: name 'plt' is not defined

In [55]:
plt.plot(averages.loc[('GEOTHERMAL',),:])

NameError: name 'plt' is not defined

In [56]:
plt.plot(averages.loc[('SOLAR PV',),:])

NameError: name 'plt' is not defined

## Pivot

Pivot is used to examine aggregates with respect to two characteristics.  You might construct a pivot of sales data if you wanted to look at average sales broken down by year and market.  


The pivot operation is essentially a `groupby` operation that transforms the rows *and the columns.*  

In [57]:
cds.pivot_table(
    values  = 'MWh', # the entry to aggregate over
    index   = 'hour',  # the row grouping attributes
    columns = 'Source',    # the column grouping attributes
    aggfunc = 'mean'   # the aggregation function
)

Source,BIOGAS,BIOMASS,GEOTHERMAL,SMALL HYDRO,SOLAR PV,SOLAR THERMAL,WIND TOTAL
hour,,,,,,,
0,225.591781,318.301370,958.720548,330.824658,0.679452,0.000000,2173.268493
1,225.964384,318.369863,959.235616,322.421918,0.643836,0.000000,2120.778082
2,225.953425,319.846575,959.367123,318.249315,0.635616,0.000000,2051.832877
3,225.887671,320.567123,958.367123,316.909589,0.419178,0.000000,1973.969863
4,225.753425,321.742466,956.347945,322.254795,0.413699,0.000000,1881.463014
5,225.243836,323.863014,956.230137,375.180822,0.482192,0.021918,1772.484932
6,224.479452,330.808219,955.682192,426.931507,352.956164,4.372603,1646.630137
7,222.454795,333.178082,953.263014,422.564384,2489.268493,58.317808,1490.194521
8,221.536986,333.936986,949.024658,376.813699,5552.531507,208.106849,1363.402740


In class challenge: create a pivot table where the columns are the hours, source is the column and the returned value is the standard deviation.

Hint: write `std` to represent standard deviation.

In [58]:
cds.pivot_table(
    values  = 'MWh',
    index   = 'Source',
    columns = 'hour',
    aggfunc = 'std'
)

hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Source,,,,,,,,,,,,,,,,,,,,,
BIOGAS,24.208922,23.991673,24.038100,24.017759,24.115212,24.141712,24.343529,24.655121,24.348700,24.293460,...,23.994552,24.065954,24.119106,24.135105,24.236869,24.395390,24.233660,24.155801,24.163106,24.181051
BIOMASS,39.395391,39.483585,39.133872,38.865774,39.296974,41.378684,43.382295,43.166284,43.748769,43.477372,...,43.339102,43.146359,44.108982,44.939578,45.188756,45.226068,45.349724,45.518096,43.297655,40.356501
GEOTHERMAL,107.008646,107.020407,106.876570,107.892763,109.751828,110.328041,110.390978,110.356101,111.295848,111.314533,...,109.817337,108.969067,107.797056,106.906617,106.493820,105.224292,103.201658,102.206035,103.147687,104.055501
SMALL HYDRO,75.704907,75.025985,76.055653,76.591467,77.159128,101.677470,99.380240,102.785461,95.406812,78.660512,...,83.461622,101.095424,113.822074,95.039306,85.177094,84.884026,90.366905,105.732538,103.713933,86.512380
SOLAR PV,8.530085,8.511417,8.498955,7.699876,7.645751,5.652271,494.037157,1466.581022,1601.608133,1518.656013,...,1850.497972,2428.163271,3084.566674,2825.992677,1601.602421,339.950077,20.858518,10.179514,9.911476,9.337124
SOLAR THERMAL,0.000000,0.000000,0.000000,0.000000,0.000000,0.330769,14.531259,86.281407,161.841097,189.281992,...,196.347549,207.831562,223.677018,202.883502,122.944332,30.685597,7.378595,4.172343,1.554107,1.412312
WIND TOTAL,1378.983785,1354.183127,1316.709390,1279.530491,1228.120679,1178.497093,1115.051493,1051.658522,1005.173453,996.708312,...,1207.036885,1267.169621,1314.250530,1339.114898,1348.444942,1359.157779,1372.515796,1385.125085,1386.860691,1389.857504
